In [5]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sym


In [14]:
x = sym.Symbol('x',real=True)
y = sym.Symbol('y',real=True)

In [15]:
def f(x,y):
    z = x + sym.I*y
    f = z**5 - 1
    f = f.expand()
    return sym.re(f),sym.im(f)



In [16]:
f0,f1 = f(x,y)
F = [f0,f1]
F

[x**5 - 10*x**3*y**2 + 5*x*y**4 - 1, 5*x**4*y - 10*x**2*y**3 + y**5]

In [17]:
J = sym.zeros(2,2)

for i in range(2):
    for j in range(2):
        if j==0:
            J[i,j] = sym.diff(F[i],x,1)
        else:
            J[i,j] = sym.diff(F[i],y,1)

J

Matrix([
[5*x**4 - 30*x**2*y**2 + 5*y**4,         -20*x**3*y + 20*x*y**3],
[         20*x**3*y - 20*x*y**3, 5*x**4 - 30*x**2*y**2 + 5*y**4]])

In [18]:
InvJ = J.inv(method='LU')
InvJ

Matrix([
[((-20*x**3*y + 20*x*y**3)*(20*x**3*y - 20*x*y**3)/((5*x**4 - 30*x**2*y**2 + 5*y**4)*(5*x**4 - 30*x**2*y**2 + 5*y**4 - (-20*x**3*y + 20*x*y**3)*(20*x**3*y - 20*x*y**3)/(5*x**4 - 30*x**2*y**2 + 5*y**4))) + 1)/(5*x**4 - 30*x**2*y**2 + 5*y**4), -(-20*x**3*y + 20*x*y**3)/((5*x**4 - 30*x**2*y**2 + 5*y**4)*(5*x**4 - 30*x**2*y**2 + 5*y**4 - (-20*x**3*y + 20*x*y**3)*(20*x**3*y - 20*x*y**3)/(5*x**4 - 30*x**2*y**2 + 5*y**4)))],
[                                                               -(20*x**3*y - 20*x*y**3)/((5*x**4 - 30*x**2*y**2 + 5*y**4)*(5*x**4 - 30*x**2*y**2 + 5*y**4 - (-20*x**3*y + 20*x*y**3)*(20*x**3*y - 20*x*y**3)/(5*x**4 - 30*x**2*y**2 + 5*y**4))),                                                            1/(5*x**4 - 30*x**2*y**2 + 5*y**4 - (-20*x**3*y + 20*x*y**3)*(20*x**3*y - 20*x*y**3)/(5*x**4 - 30*x**2*y**2 + 5*y**4))]])

In [19]:
Fn = sym.lambdify([x,y],F,'numpy')
IJn = sym.lambdify([x,y],InvJ,'numpy')

In [21]:
def NewtonRaphson(z,Fn,Jn,itmax=100,precision=1e-9):
    
    error = 1
    it = 0
    
    while error > precision and it < itmax:
        
        IFn = Fn(z[0],z[1])
        IJn = Jn(z[0],z[1])
        
        z1 = z - np.dot(IJn,IFn)
        
        error = np.max( np.abs(z1-z) )
        
        z = z1
        it +=1
        
    print(it)
    return z

In [31]:
z0 = np.array([np.cos(np.pi/5), np.sin(np.pi/5)])
zsol = NewtonRaphson(z0,Fn,IJn)
zsol

7


array([-0.80901699, -0.58778525])

In [32]:
np.cos(np.pi/5), np.sin(np.pi/5)

(0.8090169943749475, 0.5877852522924731)

---

In [6]:
# Definamos el sistema usando una lista
G = np.array([lambda x,y,z: 3*x - np.cos(y*z) - 1.5,
     lambda x,y,z: 4*x**2 - 625*y**2+2*y-1,
     lambda x,y,z: np.exp(-x*y)+20*z+(10*np.pi-3)/3])

In [7]:
def GetF(G,r):
    
    n = r.shape[0]
    
    v = np.zeros_like(r)
    
    for i in range(n):
        v[i] = G[i](r[0],r[1],r[2])
        
    return v

In [8]:
def GetJacobian(f,r,h=1e-6):
    
    n = r.shape[0]
    
    J = np.zeros((n,n))
    
    for i in range(n):
        for j in range(n):
            
            rf = r.copy()
            rb = r.copy()
            
            rf[j] = rf[j] + h
            rb[j] = rb[j] - h
            
            J[i,j] = ( f[i](rf[0],rf[1],rf[2]) - f[i](rb[0],rb[1],rb[2])  )/(2*h)
            
    
    return J

In [11]:
def NewtonRaphson(G,r,itmax=1000,error=1e-9):
    
    it = 0
    d = 1.
    dvector = []
    
    while d > error and it < itmax:
        
        # Vector actual
        rc = r
        
        F = GetF(G,rc)
        J = GetJacobian(G,rc)
        InvJ = np.linalg.inv(J)
        
        r = rc - np.dot(InvJ,F)
        
        diff = r - rc
        
        d = np.max( np.abs(diff) )
        
        dvector.append(d)

        
        it += 1
    
    print(it)
    return r,dvector

In [12]:
r,dvector = NewtonRaphson(G,np.array([0.,0.,0.]))

9


In [13]:
r

array([ 0.83319658,  0.05494366, -0.52136143])